In [1]:
# Set imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler

### Evaluating A Linear Regression Model

1 - Mean Absolute Error <br>
&emsp;&emsp;* Lets take all of the errors/residuals, sum the distance, take the average <br>
&emsp;&emsp;* This says how far off we are <br>
&emsp;&emsp;* $\displaystyle \Bigg[\frac{\sum \limits _{i=1} ^{n} |y_{i} - \hat{y_{i}}|}{n}\Bigg]$ <br><br>

2 - Mean Squared Error <br>
&emsp;&emsp;* Same concept but to punish distance from linear model more harshly <br>
&emsp;&emsp;* To do this we square the summation <br>
&emsp;&emsp;* This is good for removing outliers as they will face a more intense value weighting. <br>
&emsp;&emsp;* $\displaystyle \Bigg[\frac{\sum \limits _{i=1} ^{n} |y_{i} - \hat{y_{i}}|^{2}}{n}\Bigg]$ <br><br>

3 - Root Mean Squared Error <br>
&emsp;&emsp;* Same concept but to punish distance from linear model more harshly <br>
&emsp;&emsp;* To do this we square the summation <br>
&emsp;&emsp;* This is good for more direct comparison to Y. The root mean squared provides an easier y comparion <br>
&emsp;&emsp;* $\displaystyle \sqrt{\Bigg[\frac{\sum \limits _{i=1} ^{n} |y_{i} - \hat{y_{i}}|^{2}}{n}\Bigg]}$<br><br>

4 - Coefficient of Determination <br>
&emsp;&emsp;* RSS - Sum of Squared Residuals -> The Numerator in the Above Functions<br>
&emsp;&emsp;* TSS - Total Sum of Squares<br><br>
&emsp;&emsp;* TSS = $\displaystyle \sum \limits _{i=1} ^{n} |y_{i} - \bar{y_{i}}|^{2}$<br><br>
&emsp;&emsp;* Notice Instead of predicted value '^' we use average/mean value '-'<br>
&emsp;&emsp;* A value close to 0 with $R^{2}$ is a good indicator usually<br><br>
&emsp;&emsp;* $R^{2} = 1 - \Bigg[\frac{RSS}{TSS}\Bigg]$ <br><br>
